## Explainable Boosting Machine (一般化加法モデル) による解釈性の高いモデル開発
製造プロセスから採取されたセンサーデータと検査結果のデータを用いて、品質管理モデルを構築します。[interpretml - interpret](https://github.com/interpretml/interpret) に含まれる一般化加法モデルの推定アルゴリズム Explainable Boosting Machine を用います。

### 0. 事前準備
- Jupyter Kernel :  `rai-ebm` を選択する。
    - [0-Setup.ipynb](./0-Setup.ipynb) の手順に従い構築しておくこと。
- IDE : JupyterLab, Jupyter Notebook を推奨します。
    - VSCode が widgets に対応していないためです。
    

### 1. データ準備
データを Pandas DataFrame としてインポートします。

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from helper import ebm_preserve_hist, ebm_preserve_global, ebm_preserve_local

# 製造工程のサンプルのダミーデータ
df = pd.read_csv("../data/Factory.csv")

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

# 説明変数の選択
X = df.drop(columns=["Quality","ID"],axis=1)

# 目的変数の選択
y = df["Quality"].values

# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=100, stratify=y)

### 2. データ探索
InterpreML にはデータの可視化のためのメソッドも組み込まれています。

In [ ]:
from interpret import show
from interpret.data import ClassHistogram

hist = ClassHistogram().explain_data(X_train, y_train, name = 'Train Data')
show(hist) # 表示されない場合はセル実行を再試行してください。

In [ ]:
# # for vscode user
# ebm_preserve_hist(hist, 'hist')

### 3. Explainable Boosting Machine (EBM) によるモデル学習
解釈性の高い EBM アルゴリズムによって一般化加法モデルの推定を行います。交互作用項を考慮したいときは `interactions` で組み合わせ数や列インデックス番号の組み合わせを指定します。

In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier, LogisticRegression, ClassificationTree, DecisionListClassifier
seed = 1234
ebm = ExplainableBoostingClassifier(random_state=seed, interactions=4)
ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays

### 4. モデル解釈 (グローバル)
モデルの重要度と各説明変数や交互作用項の予測値に対する寄与度のグラフを表示します。

In [ ]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

In [ ]:
# # for vscode user
# ebm_preserve_global(ebm_global, 'ebm_global')
# ebm_preserve_local(ebm_global, 'ebm_global')

### 5. モデル解釈 (ローカル)
学習済みの EBM モデルから算出されたテストデータの個々の予測値の解釈を行います。

In [ ]:
# 例として、テストデータの冒頭 20 個のデータを対象
ebm_local = ebm.explain_local(X_test[:20], y_test[:20], name='EBM')
show(ebm_local)

In [ ]:
# # for vscode user
# ebm_preserve_local(ebm_local, 'ebm_local')

###  6. EBM モデルの精度確認
モデル精度を ROC で確認します。

In [ ]:
from interpret.perf import ROC

ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)

In [ ]:
# # for vscode user
# ebm_preserve_global(ebm_perf, 'ebm_perf')

### 7. ロジスティック回帰、決定木モデルの学習

In [ ]:
from interpret.glassbox import LogisticRegression, ClassificationTree

X_enc = pd.get_dummies(X, prefix_sep='.')
feature_names = list(X_enc.columns)
X_train_enc, X_test_enc, y_train, y_test = train_test_split(X_enc, y, test_size=0.20, random_state=seed)

lr = LogisticRegression(random_state=seed, feature_names=feature_names, penalty='l2')
lr.fit(X_train_enc, y_train)

tree = ClassificationTree()
tree.fit(X_train_enc, y_train)

### 8. ダッシュボードでのモデル比較

In [ ]:
lr_perf = ROC(lr.predict_proba).explain_perf(X_test_enc, y_test, name='Logistic Regression')
tree_perf = ROC(tree.predict_proba).explain_perf(X_test_enc, y_test, name='Classification Tree')

show(lr_perf)
show(tree_perf)
show(ebm_perf)

In [ ]:
# ebm_preserve_global(lr_perf, 'lr_perf')
# ebm_preserve_global(tree_perf, 'tree_perf')

### 9. モデル解釈の比較 (グローバル、ローカル)

#### Global Explain

In [ ]:
lr_global = lr.explain_global(name='LR')
tree_global = tree.explain_global(name='Tree')

show(lr_global)
show(tree_global)
show(ebm_global)

In [ ]:
# # Decision Tree は Dash を利用しているが、Intepret (version 0.2.5) が Dash の保存をまだサポートしていない
# ebm_preserve_global(lr_global, 'lr_global')
# ebm_preserve_local(lr_global, 'lr_global')


#### Local Explain

In [ ]:
lr_local = lr.explain_local(X_test[:20], y_test[:20], name='LR')
tree_local = tree.explain_local(X_test[:20], y_test[:20], name='Tree')

show(lr_local)
show(tree_local)
show(ebm_local)

In [ ]:
# # Decision Tree は Dash を利用しているが、Intepret (version 0.2.5) が Dash の保存をまだサポートしていない
# ebm_preserve_local(lr_local, 'lr_local')

#### ダッシュボード
※ interpret バージョン 0.2.5 時点では、このダッシュボード機能はクラウド環境で利用できません。

In [ ]:
# EBM、ロジスティック回帰、決定木の比較を行うダッシュボード
show([hist, lr_global, lr_local, lr_perf, tree_global, tree_local, tree_perf, ebm_global, ebm_local, ebm_perf], share_tables=True)